(Visit the
[documentation](https://datafold-dev.gitlab.io/datafold/tutorial_index.html) page
to view the executed notebook.)

# Model predictive control: Inverted pendulum
In this tutorial we introduce the usage of the Dynamic Mode Decomposition (DMD) method with controlled systems. Furthermore, we demonstrate how the Extended-DMD can be used within model predictive control.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [ ]:
from datafold import (
    EDMD,
    DMDControl,
    InverseQuadraticKernel,
    TSCDataFrame,
    TSCIdentity,
    TSCRadialBasis,
)
from datafold.appfold.kmpc import AffineKgMPC, LinearKMPC
from datafold.utils._systems import InvertedPendulum

## Data generation

### Inverted pendulum physics
A test model implemented in datafold is used to generate example data in this tutorial. A pendulum and a moving cart are connected by a swivel, which allows the pendulum to freely rotate. The cart wheels spin on a rail, and the entire system is powered by a DC motor. The displacement of the cart $x$ and the angular rotation of the pendulum $\theta$ describe the movement of the pendulum (note that the time derivatives, $\dot x$ and $\dot \theta$, are also needed to describe the full state), and the voltage to the motor $u$ can be controlled.

### Creating the training set
The training set will consists of 20 different trajectories of 10 seconds each, discretized at 0.01 s time steps. The initial conditions of all trajectories is the same, and the control input is a sinusoidal signal of varying amplitude, frequency and phase. The 20 timeseries are concatenated into datafold's data structure for handling time series collections, `TSCDataFrame`.

In [ ]:
# Data generation parameters
training_size = 20
time_values = np.arange(0, 10 + 1e-15, 0.01)

# Sample from a single initial condition (but use randomly sampled control signals below)
ic = np.array([0, 0, 0, 0])

In [ ]:
X_tsc, U_tsc = [], []  # lists to collect sampled time series

rng = np.random.default_rng(1)  # seed to repeat evaluation

for i in range(training_size):
    control_amplitude = 0.1 + 0.9 * rng.uniform(0, 1)
    control_frequency = np.pi + 2 * np.pi * rng.uniform(0, 1)
    control_phase = 2 * np.pi * rng.uniform(0, 1)
    Ufunc = lambda t, y: control_amplitude * np.sin(
        control_frequency * t + control_phase
    )

    invertedPendulum = InvertedPendulum()

    # X are the states and U is the control input acting on the state's evolution
    X, U = invertedPendulum.predict(
        X=ic,
        Ufunc=Ufunc,
        time_values=time_values,
    )

    X_tsc.append(X)
    U_tsc.append(U)


# Turn lists into time series collection data structure
X_tsc = TSCDataFrame.from_frame_list(X_tsc)
U_tsc = TSCDataFrame.from_frame_list(U_tsc);

We can look at the sampled time series data (`X_tsc`) on the state space and the control (`U_tsc`) that acted on the system evolution.

In [ ]:
plt.figure(figsize=(16, 7))

plt.subplot(211)
plt.title(
    r"Training data - cart position $\theta$ $\mathbf{x}$ (each time series is separated by verical lines)"
)
plt.plot(X_tsc["x"].to_numpy())
[plt.axvline(i, c="black") for i in np.arange(0, X_tsc.shape[0], X_tsc.n_timesteps)]
plt.xticks([])


plt.subplot(212)
plt.title(
    r"Training data - control voltage $\mathbf{u}$ (each time series is separated by verical lines)"
)
plt.plot(U_tsc["u"].to_numpy())
[plt.axvline(i, c="black") for i in np.arange(0, U_tsc.shape[0], U_tsc.n_timesteps)]
plt.xticks([]);

We store the last time series as a system reference to use throughout the tutorial. We first look at a snapthot of the numeric values in the data frames of `X_tsc` and `U_tsc`.

In [ ]:
X_last = X_tsc.loc[[19], :]
U_last = U_tsc.loc[[19], :]

X_last

In [ ]:
U_tsc

## Identifying the pendulum system 

### Dynamic mode decomposition with contol
In this first part we demonstrate how to use the `DMDControl` class to create a predictor based on the dynamic mode decomposition for controlled systems using the generated data from the inverted pendulum.

In datafold the `DMDControl` class uses the `TSCPredictMixin` and is therefore provides a `scikit-learn` estimator-style interface (generalized for time series predictions and control input). After the model is initialized, we can use the `.fit` method to train the model, using the time series collection data structure (`TSCDataFrame`). Internally, the `DMDControl` computes two matrices $A$ and $B$, which best satisfy $\mathbf{x}_{k+1} = A\mathbf{x}_k + B\mathbf{u}_k$ ($\mathbf{x}$ refering to the state vector of the system and $\mathbf{u}$ to the control input). The model assumes that the system is linear both in the state and control input.  

In [ ]:
dmdc = DMDControl()
dmdc.fit(X_tsc, U=U_tsc);

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("state matrix $A$")
plt.imshow(dmdc.sys_matrix_)
plt.colorbar()

plt.subplot(122)
plt.title("control matrix $B$")
plt.imshow(dmdc.control_matrix_)
plt.colorbar();

The `.predict` method can be used to estimate the trajectory of the system state, given an initial condition and the control input over the prediction horizon.

In [ ]:
prediction = dmdc.predict(ic, U=U_last)

In [ ]:
plt.figure(figsize=(16, 3))

plt.subplot(121)
plt.title(r"Linear DMD prediction - cart position $x$")
plt.plot(time_values, prediction["x"].to_numpy(), label="prediction")
plt.plot(time_values, X["x"].to_numpy(), label="actual")
plt.legend()

plt.subplot(122)
plt.title(r"EDMD(100 random rbf) prediction - pendulum angle $\theta$")
plt.plot(time_values, prediction["theta"].to_numpy(), label="prediction")
plt.plot(time_values, X_last["theta"].to_numpy(), label="actual")
plt.legend();

### Using DMDControl within the Extended Dynamic Mode Decomposition (EDMD) model framework

A `EDMD` class has two main arguments

* `dict_steps` (dictionary): corresponding to a set of functions with the goal to approximately linearize the system dynamics
* `dmd_model` (final estimator): corresponding to a DMD model to performs a linear system identification in the lifted space

(see also the dedicated `EDMD` tutorial)

For the `dict_steps` we use a dictionary of 100 randomly selected radial basis functions (RBF) specified with an inverse quadratic kernel. The functions are centered at random positions on the state space. 

For the `dmd_model` parameter, we can set a DMD variant that supports control input (such as `DMDControl`). Essentially, this makes the `EDMD` model to also support controlled systems.


#### Select RBF centers

In [ ]:
n_rbf_centers = 100
eps = 1

rbf = TSCRadialBasis(
    kernel=InverseQuadraticKernel(epsilon=eps), center_type="fit_params"
)
center_ids = sorted(
    np.random.choice(range(0, 1000 * training_size), size=n_rbf_centers, replace=False)
)
centers = X_tsc.iloc[center_ids].to_numpy()

#### Setting up `EDMD` with radial basis function dictionary and `DMDControl`

We can now set up and train the `EDMD` model. 

Note that using `TSCRadialBasis` transformer with `center_type='fit_params'` requires passing the `centers` parameter to the `.fit` method of transformer using the `fit_params` as detailed in the documentation.

In [ ]:
edmdrbf = EDMD(dict_steps=[("rbf", rbf)], dmd_model=DMDControl(), include_id_state=True)

edmdrbf.fit(
    X_tsc,
    U=U_tsc,
    rbf__centers=centers,
)

rbfprediction = edmdrbf.predict(ic, U=U_last)

In [ ]:
plt.figure(figsize=(16, 3))
plt.subplot(121)
plt.title(r"EDMD(100 random rbf) prediction - cart position $x$")
plt.plot(time_values, rbfprediction["x"].to_numpy(), label="prediction")
plt.plot(time_values, X_last["x"].to_numpy(), label="actual")
plt.legend()

plt.subplot(122)
plt.title(r"EDMD(100 random rbf) prediction - pendulum angle $\theta$")
plt.plot(time_values, rbfprediction["theta"].to_numpy(), label="prediction")
plt.plot(time_values, X_last["theta"].to_numpy(), label="actual")
plt.legend();

In the figure above we can see the the EDMD using the linear DMD estimator predicts the displacement even better than the DMD, but the prediction for $\theta$ is good only on a short time scale.

## Koopman Model Predictive Control

Model Predictive Control (MPC) is a method for estimating a control signal to provide to a system to achieve certain behaviour in the future. 

An important parameter for MPC is the prediction horizon, which denotes how far in the future is the control signal computed. Here we set it to 100 timesteps (corresponding to one second). The optimality of the signal is computed using a cost function of the referenced states (`qois`) and the control signal itself. Here we provide both $\mathbf{x}$ and $\theta$ as referenced states, but set the running cost weight of $\theta$ to 0 to showcase the functionality of the interface. Additionaly, the cost function enables penalizing control inputs. To match the test data as well as possible, this is set to a small value.

#### Linear MPC
Here we show an implementation of a Koopman MPC (KMPC) where the model part is based on the Koopman operator. The `LinearKMPC` class implements such a controller based on `EDMD` (as used above with `dmd_model=DMDControl()`). The key benefit is that the model is a linear system in the lifted space and the optimal control can be directly computed using a quadratic programming optimizer.

In [ ]:
horizon = 100  # in time steps

kmpc = LinearKMPC(
    predictor=edmdrbf,
    horizon=horizon,
    state_bounds=np.array([[1, -1], [6.28, 0]]),
    input_bounds=np.array([[5, -5]]),
    qois=["x", "theta"],
    cost_running=np.array([100, 0]),
    cost_terminal=1,
    cost_input=0.001,
)

To generate the control signal, a full initial state is required, in addition to a reference to track. Here we use a reference produced by a known control signal in the training data, to be able to later compare it to the optimal control signal computed by the controller.

In [ ]:
reference = X_last[["x", "theta"]].iloc[: horizon + 1]
ukmpc = kmpc.generate_control_signal(rbfprediction.initial_states(), reference)

Furthermore, we compute what trajectory the model has predicted based on the control signal computed by the controller and what is the real response of the system.

In [ ]:
kmpcpred = edmdrbf.predict(rbfprediction.initial_states(), U=ukmpc)

ukmpc_interp = interp1d(
    rbfprediction.time_values()[:horizon],
    ukmpc,
    axis=0,
    fill_value=0,
    bounds_error=False,
)

ukmpc_func = lambda t, y: ukmpc_interp(t).T
kmpctraj, _ = invertedPendulum.predict(
    X=ic,
    Ufunc=ukmpc_func,
    time_values=time_values,
)

kmpctraj

We now plot the data that we obtained from the `KoopmanMPC` class and compare the predicted trajectories for both position $x$ and angle $\theta$. In addition, we also compare the estimated control signal from the `KoopmanMPC` class and compare it with the the reference control signal above.

In [ ]:
plt.figure(figsize=(16, 3.5))

plt.subplot(131)
plt.title(r"Comparison : Cart Position $x$")
plt.plot(
    time_values[: horizon + 1], X_last["x"].to_numpy()[: horizon + 1], label="reference"
)
plt.plot(
    time_values[: horizon + 1],
    kmpcpred["x"].to_numpy()[: horizon + 1],
    label="prediction",
)
plt.plot(
    time_values[: horizon + 1], kmpctraj["x"].to_numpy()[: horizon + 1], label="actual"
)
plt.legend()

plt.subplot(132)
plt.title(r"Comparison : Pendulum Angle $\theta$")
plt.plot(
    time_values[: horizon + 1],
    X_last["theta"].to_numpy()[: horizon + 1],
    label="reference",
)
plt.plot(
    time_values[: horizon + 1],
    kmpcpred["theta"].to_numpy()[: horizon + 1],
    label="prediction",
)
plt.plot(
    time_values[: horizon + 1],
    kmpctraj["theta"].to_numpy()[: horizon + 1],
    label="actual",
)
plt.legend()

plt.subplot(133)
plt.title(r"Comparison : Control Signal $u$")
plt.plot(time_values[: horizon + 1], U_last.to_numpy()[: horizon + 1], label="correct")
plt.plot(time_values[:horizon], ukmpc, label="controller")
plt.legend();
